In [1]:
from keras.models import Model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D
import h5py
import numpy as np
from scipy.misc import imsave
from keras import backend as K

img_width, img_height = 128, 128

Using Theano backend.
Using cuDNN version 5103 on context None
Mapped name None to device cuda: Graphics Device (0000:07:00.0)


In [2]:
# build the VGG16 network
input_img = Input(shape=(3, img_width, img_height))
layers = ZeroPadding2D((1, 1))(input_img)
layers = Convolution2D(64, (3, 3), activation='relu', name='conv1_1')(layers)
layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(64, (3, 3), activation='relu', name='conv1_2')(layers)
layers = MaxPooling2D((2, 2), strides=(2, 2))(layers)

layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(128, (3, 3), activation='relu', name='conv2_1')(layers)
layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(128, (3, 3), activation='relu', name='conv2_2')(layers)
layers = MaxPooling2D((2, 2), strides=(2, 2))(layers)

layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(256, (3, 3), activation='relu', name='conv3_1')(layers)
layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(256, (3, 3), activation='relu', name='conv3_2')(layers)
layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(256, (3, 3), activation='relu', name='conv3_3')(layers)
layers = MaxPooling2D((2, 2), strides=(2, 2))(layers)

layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(512, (3, 3), activation='relu', name='conv4_1')(layers)
layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(512, (3, 3), activation='relu', name='conv4_2')(layers)
layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(512, (3, 3), activation='relu', name='conv4_3')(layers)
layers = MaxPooling2D((2, 2), strides=(2, 2))(layers)

layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(512, (3, 3), activation='relu', name='conv5_1')(layers)
layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(512, (3, 3), activation='relu', name='conv5_2')(layers)
layers = ZeroPadding2D((1, 1))(layers)
layers = Convolution2D(512, (3, 3), activation='relu', name='conv5_3')(layers)
last_layer = MaxPooling2D((2, 2), strides=(2, 2))(layers)


model = Model(inputs=input_img, outputs=last_layer)

# get the symbolic outputs of each "key" layer (we gave them unique names).
layer_dict = dict([(layer.name, layer) for layer in model.layers])

In [3]:
weights_path = 'vgg16_weights.h5'

f = h5py.File(weights_path)
for k in range(f.attrs['nb_layers']):
    if k >= len(model.layers)-1:
        # we don't look at the last (fully-connected) layers in the savefile
        break
    g = f['layer_{}'.format(k)]
    weights = [g['param_{}'.format(p)] for p in range(g.attrs['nb_params'])]
    if len(weights) == 2:
        weights[0] = np.transpose(weights[0], (2, 3, 1, 0))
    model.layers[k+1].set_weights(weights)
f.close()
print('Model loaded.')

Model loaded.


In [4]:
# util function to convert a tensor into a valid image
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [5]:
layer_name = 'conv3_1'
filter_index = 5  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = layer_dict[layer_name].output
loss = K.mean(layer_output[:, filter_index, :, :])

# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, input_img)[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_img], [loss, grads])

In [6]:
# we start from a gray image with some noise
input_img_data = np.random.random((1, 3, img_width, img_height)) * 20 + 128.
step = 20

img = input_img_data[0]
img = deprocess_image(img)
imsave('%s_filter_%d_before.png' % (layer_name, filter_index), img)

# run gradient ascent for 20 steps
for i in range(20):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += grads_value * step
    
img = input_img_data[0]
img = deprocess_image(img)
imsave('%s_filter_%d_after.png' % (layer_name, filter_index), img)

In [7]:
from IPython.display import Image
Image(url= "conv3_1_filter_5_before.png")

In [8]:
Image(url= "conv3_1_filter_5_after.png")